In [36]:
import os
import re

import sys
sys.path.append("..")

import time
import pickle
import graphviz
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.svm import SVR

from scripts.clean_results import clean_results_df, dump_results
from scripts.plotting_functions import export_tree_graph, render_tree_graph
from scripts.plotting_functions import plot_feature_importance, plot_coefficients_significance,plot_models_performance

from scripts.analyzers import simple_stemmer, regexp_stemmer

from bokeh.plotting import output_notebook, show
output_notebook()

SEED = 99
np.random.seed(SEED)

DATA_FOLDER = os.path.join(os.getcwd(), "../data")
RESULTS_FOLDER = os.path.join(DATA_FOLDER, "results")
CV_RESULTS_FOLDER = os.path.join(RESULTS_FOLDER, "csv")

Loading BokehJS ...

<h3>Lettura dati di training &amp; testing</h3>

In [2]:
train_fp = os.path.join(os.getcwd(), "../data/csv/train.csv")
train = pd.read_csv(
    train_fp,
    index_col=['review_id']
)

In [3]:
test_fp = os.path.join(os.getcwd(), "../data/csv/train.csv")
test = pd.read_csv(
    test_fp,
    index_col=['review_id']
)

In [4]:
X_train, Y_train = train['text'], train['stars']
X_test, Y_test = test['text'], test['stars']

<h3>Migliori parametri per modello SGD</h3>

In [5]:
sgd_cv_results_fp = os.path.join(CV_RESULTS_FOLDER, "ngrams_sgd_cv_results.csv")
sgd_cv_results_df = pd.read_csv(sgd_cv_results_fp, index_col="rank_test_neg_root_mean_squared_error")

In [6]:
sgd_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2), norm='l2', smooth_idf=True)),
    ('model', SGDRegressor(alpha=0.00001, penalty='l2'))
])

In [7]:
sgd_pipeline.fit(X_train, Y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
                ('model', SGDRegressor(alpha=1e-05))])

In [8]:
sgd_coefficients = sgd_pipeline.steps[1][1].coef_

In [9]:
len(sgd_coefficients)

1498685

In [10]:
sgd_features_names = sgd_pipeline.steps[0][1].vocabulary_

In [11]:
len(sgd_features_names)

1498685

In [12]:
sgd_coeff_names = []
for name in sgd_features_names:
    index = sgd_features_names[name]
    sgd_coeff_names.append((name, sgd_coefficients[index], abs(sgd_coefficients[index])))

In [13]:
sgd_coeff_names.sort(reverse=True, key=lambda t : t[2])

In [14]:
n = 35
sgd_importance_fig = plot_coefficients_significance(
    sgd_coeff_names,
    n,
    title="35 features più importanti per SGDRegressor"
)

In [15]:
show(sgd_importance_fig)

In [16]:
sgd_ngrams_coeff_names = [t for t in sgd_coeff_names if len(t[0].split()) > 1]

In [17]:
n = 20
sgd_ngrams_importance_fig = plot_coefficients_significance(
    sgd_ngrams_coeff_names,
    n,
    title="20 bi-grammi più importanti per SGDRegressor"
)

In [18]:
show(sgd_ngrams_importance_fig)

<h3>Migliori parametri per modello DTR</h3>

In [19]:
dtr_cv_results_fp = os.path.join(CV_RESULTS_FOLDER, "ngrams_dtr_cv_results.csv")
dtr_cv_results_df = pd.read_csv(dtr_cv_results_fp, index_col="rank_test_neg_root_mean_squared_error")

In [20]:
dtr_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2), norm='l2', smooth_idf=False)),
    ('model', DecisionTreeRegressor(
        max_depth=70,
        max_features=7000,
        min_samples_leaf=400,
        min_samples_split=1000,
        splitter='random')
    )
])

In [21]:
dtr_pipeline.fit(X_train, Y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(ngram_range=(1, 2), smooth_idf=False)),
                ('model',
                 DecisionTreeRegressor(max_depth=70, max_features=7000,
                                       min_samples_leaf=400,
                                       min_samples_split=1000,
                                       splitter='random'))])

In [22]:
dtr_features_names = (dtr_pipeline.steps[0][1].vocabulary_)

In [23]:
len(dtr_features_names)

1498685

In [24]:
dtr_features_importance = dtr_pipeline.steps[1][1].feature_importances_

In [25]:
len(dtr_features_importance)

1498685

In [26]:
export_tree_graph(
    decision_tree = dtr_pipeline.steps[1][1],
    vocabulary = dtr_features_names,
    out_file = "ngrams_tree.dot"
)

In [27]:
render_tree_graph("ngrams_tree.dot")

<img src="./imgs/ngrams_tree.png" style="width=800;height=600">

In [26]:
dtr_importance_names = []
for name in dtr_features_names:
    index = dtr_features_names[name]
    if(dtr_features_importance[index] > 0):
        dtr_importance_names.append((name, dtr_features_importance[index]))
        
dtr_importance_names.sort(reverse=True, key=lambda t : t[1])

In [27]:
n = 25
dtr_importance_fig = plot_feature_importance(
    [t[0] for t in dtr_importance_names[:n]],
    [t[1] for t in dtr_importance_names[:n]],
    title="Importanza migliori {} features Decision Tree Regressor".format(n),
)
show(dtr_importance_fig)

<h3>Migliori parametri per modello SVR</h3>

In [36]:
svr_cv_results_fp = os.path.join(CV_RESULTS_FOLDER, "ngrams_svr_cv_results.csv")
svr_cv_results_df = pd.read_csv(svr_cv_results_fp, index_col="rank_rmse")

In [29]:
svr_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2), norm='l2', smooth_idf=False)),
    ('model', SVR(C=50, kernel='linear', gamma='scale'))
])

In [30]:
svr_pipeline.fit(X_train, Y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(ngram_range=(1, 2), smooth_idf=False)),
                ('model', SVR(C=50, kernel='linear'))])

In [39]:
svr_features_names = svr_pipeline.steps[0][1].vocabulary_

In [40]:
len(svr_features_names)

1498685

In [44]:
svr_coeffs = svr_pipeline.steps[1][1].coef_.todense()

In [62]:
svr_coeffs.shape[1]

1498685

In [63]:
svr_coeff_names = []
for name in svr_features_names:
    index = svr_features_names[name]
    svr_coeff_names.append((name, svr_coeffs[0, index], abs(svr_coeffs[0, index])))

In [66]:
svr_coeff_names.sort(reverse=True, key=lambda t : t[2])

In [69]:
n = 25
svr_importance_fig = plot_coefficients_significance(
    svr_coeff_names,
    n,
    title="Migliori Coefficienti modello SVR kernel lineare")

In [70]:
show(svr_importance_fig)

<h3>Comparazione errori di training</h3>
<p>Procediamo ora con il confrontare l'errore quadratico medio dei vari modelli al fine di selezionare il modello più performante. Successivamente testeremo la capacità del miglior modello di generalizzare. Per fare questo sfrutteremo l'<i>hold-out</i> set creato nel primo notebook.<br>
Potremmo essere tentanti di testare tutti i modelli ottenuti sull'insieme precedentemente menzionato e selezionare il modello che mostra l'errore più piccolo su tale insieme. Questa procedura va però evitata in quanto si rischia di overfittare i modelli alle particolari caratteristiche dell'insieme di test.

<p>Visto che abbiamo precedentemente ottenuto, tramite validazione incrociata, le misure di errore per i vari modelli ed avendole memorizzate in dei files è ora sufficiente metterle a confronto. Una volta selezionato il miglior modello procederemo ad addestrarlo sull'intero insieme di training e testarlo sull'insieme di test.

In [37]:
best_models_df = pd.read_csv(
    os.path.join(RESULTS_FOLDER,'best_models_ranking.csv'),
    index_col=['rank_rmse', 'rank_mae']
)
best_models_df.loc[:, ['model','ngram_range','mean_rmse','std_rmse','mean_mae','std_mae']]

,,model,ngram_range,mean_rmse,std_rmse,mean_mae,std_mae
rank_rmse,rank_mae,,,,,,
1.0,4.0,SVR(),"(1, 2)",0.821007,0.005164,0.653451,0.005016
2.0,3.0,SGDRegressor(),"(1,1)",0.829567,0.005108,0.652813,0.003873
3.0,5.0,SGDRegressor(),"(1, 2)",0.945281,0.004997,0.767041,0.004767
4.0,1.0,DecisionTreeRegressor(),"(1,1)",1.283854,0.028661,1.045344,0.031268
5.0,6.0,DecisionTreeRegressor(),"(1, 2)",1.465017,0.020844,1.248204,0.026967
6.0,2.0,LinearRegression(),"(1,1)",1.466439,0.026019,0.989667,0.009860


In [38]:
models_performance_fig = plot_models_performance(best_models_df)

In [39]:
show(models_performance_fig)

<h3>Valutazione modello sull'insieme di test</h3>
<p>Ora che abbiamo un'idea dell'errore commesso dai modelli sull'insieme di training e necessario valutarne la capacità di generalizzare a nuove istanze non incontrate durante la fase di addestramento.<br>

In [31]:
X_test_transformed = svr_pipeline.steps[0][1].transform(X_test)
validation_preds = svr_pipeline.steps[1][1].predict(X_test_transformed)
validation_mse = mean_squared_error(Y_test, validation_preds)

In [32]:
validation_rmse = np.sqrt(validation_mse)

In [33]:
validation_rmse

0.09676326798812296

<p>Sembrerebbe che le predizioni del modello sull'insieme di test siano particolarmente accurate. E' strano che l'errore di validazione sia così inferiore a quello di training, fenomeno che non mi sono ancora riuscita a spiegare.
<p>Nonostante sia generalmente accettato che l'insieme di test non vada utilizzato per validare più di un modello (in quanto si rischia di selezionare il modello che meglio si adatta alle particolari caratteristiche dell'insieme di test) lo utilizzeremo per validare anche uno dei modelli lineari per verificare se anche in quel caso l'errore di validazione sia di parecchio inferiore a quello di test.

In [364]:
sgd_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2), norm='l2', smooth_idf=True)),
    ('model', SGDRegressor(alpha=0.00001, penalty='l2'))
])
sgd_pipeline.fit(X_train, Y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
                ('model', SGDRegressor(alpha=1e-05))])

In [365]:
X_test_transformed_2 = sgd_pipeline.steps[0][1].transform(X_test)
validation_preds_2 = sgd_pipeline.steps[1][1].predict(X_test_transformed_2)
validation_mse_2 = mean_squared_error(Y_test, validation_preds_2)

In [366]:
validation_rmse_2 = np.sqrt(validation_mse_2)

In [367]:
validation_rmse_2

0.9165432767470858

<p>In questo caso l'errore di validazione del modello SGDRegressor è poco maggiore dell'errore di training. Questo è abbastanza comune ma non troppo preoccupante in quanto la discrepanza è abbastanza piccola.